# Otto RecSys - Candidate ReRank Model - Embedding Session using LightGCN

In [1]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.0 MB/s eta 0:00:00


In [2]:
# Set this seed
SEED = 1023

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import polars as pl
import numpy as np
from tqdm.notebook import tqdm
import random
import gc
import os

# Thiết lập seed để có thể tái lập kết quả
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(SEED)

In [4]:
def load_raw_data_parquet(file_pattern):
    return pl.scan_parquet(file_pattern) \
        .with_columns(pl.col('type').replace(type_map).cast(pl.Int8)) \
        .with_columns((pl.col('ts') / 1000).cast(pl.Int32)) \
        .with_columns(pl.col('aid').cast(pl.Int32)) \
        .collect()

In [5]:
type_map = {
    'clicks': 0,
    'carts': 1,
    'orders': 2,
}

In [6]:
full_train_df = load_raw_data_parquet('/kaggle/input/otto-validation/train_parquet/*')

In [7]:
# Cell 2 (PHIÊN BẢN SỬA LỖI - SESSION SAMPLING)

print("--- Step 1: Preparing Data (with Session Sampling) ---")

# --- LẤY MẪU SESSION ---
TRAIN_SESSION_SAMPLE_RATE = 0.05 # Bắt đầu với một con số nhỏ, ví dụ 5%
print(f"Sampling {TRAIN_SESSION_SAMPLE_RATE*100}% of unique sessions...")

# 1. Lấy danh sách tất cả các session ID duy nhất
all_train_sessions = full_train_df['session'].unique()
print(f"Total unique sessions in full train data: {len(all_train_sessions)}")

# 2. Lấy mẫu trên danh sách ID này
sampled_train_sessions = all_train_sessions.sample(fraction=TRAIN_SESSION_SAMPLE_RATE, shuffle=True, seed=SEED)
print(f"Number of sessions after sampling: {len(sampled_train_sessions)}")

# 3. Lọc lại DataFrame gốc để chỉ giữ lại các session đã được lấy mẫu
train_data = full_train_df.filter(pl.col('session').is_in(sampled_train_sessions))
print(f"Shape of the final training data: {train_data.shape}")

del full_train_df, all_train_sessions, sampled_train_sessions
gc.collect()


--- Step 1: Preparing Data (with Session Sampling) ---
Sampling 5.0% of unique sessions...
Total unique sessions in full train data: 11098528
Number of sessions after sampling: 554926
Shape of the final training data: (8212182, 4)


30

In [8]:
print("\n--- Creating Mappings for User and Item Nodes ---")

# Lấy ra tất cả các session và aid duy nhất
unique_sessions = train_data['session'].unique()
unique_aids = train_data['aid'].unique()

# Đếm số lượng node
num_sessions = len(unique_sessions)
num_aids = len(unique_aids)
print(f"Number of unique sessions (users): {num_sessions}")
print(f"Number of unique AIDs (items): {num_aids}")


--- Creating Mappings for User and Item Nodes ---
Number of unique sessions (users): 554926
Number of unique AIDs (items): 895417


In [9]:
session2idx = {session_id: i for i, session_id in enumerate(unique_sessions)}

In [10]:
list(session2idx.keys())[:10]

[8, 10, 21, 25, 104, 110, 113, 148, 152, 173]

In [11]:
aid2idx = {aid: i + num_sessions for i, aid in enumerate(unique_aids)}
list(aid2idx.keys())[:10]

[0, 3, 4, 10, 12, 14, 16, 17, 20, 21]

In [12]:
# Tạo các từ điển ngược để tham chiếu sau này (tùy chọn nhưng hữu ích)
idx2session = {i: session_id for session_id, i in session2idx.items()}
idx2aid = {i: aid for aid, i in aid2idx.items()}

In [13]:
print("\n--- Creating Edge List for the Bipartite Graph ---")

# Lấy các cặp (session, aid) duy nhất từ dữ liệu
# Đây chính là các cạnh của đồ thị hai phía của chúng ta
interactions = train_data.select(['session', 'aid']).unique()
print(f"Found {len(interactions)} unique interactions (edges).")


--- Creating Edge List for the Bipartite Graph ---
Found 5119526 unique interactions (edges).


In [14]:
# Áp dụng các từ điển ánh xạ để chuyển đổi ID sang index
print("Applying mappings to create edge indices...")

# Sử dụng .map_dict() của Polars để chuyển đổi hiệu quả
source_nodes = interactions['session'].replace(session2idx)
destination_nodes = interactions['aid'].replace(aid2idx)

Applying mappings to create edge indices...


In [15]:
# Tạo edge_index tensor
# PyTorch Geometric yêu cầu định dạng [2, num_edges]
edge_index_numpy = np.array([
    source_nodes.to_numpy(),
    destination_nodes.to_numpy()
])

# Đồ thị GNN là vô hướng (undirected), nên chúng ta cần thêm các cạnh theo chiều ngược lại
reverse_edge_index_numpy = np.array([
    destination_nodes.to_numpy(),
    source_nodes.to_numpy()
])

# Nối cả hai chiều lại
full_edge_index_numpy = np.concatenate([edge_index_numpy, reverse_edge_index_numpy], axis=1)

# Chuyển sang PyTorch tensor
edge_index = torch.LongTensor(full_edge_index_numpy)

print("\n--- Edge List Creation Complete! ---")
print(f"Shape of the final edge_index tensor: {edge_index.shape}")
print("Example edges (first 5):")
print(edge_index[:, :5])


--- Edge List Creation Complete! ---
Shape of the final edge_index tensor: torch.Size([2, 10239052])
Example edges (first 5):
tensor([[ 117461,  214562,  327001,   47744,  173587],
        [1234288, 1348115,  909818, 1370867, 1240861]])


In [16]:
print("--- Step 2: Building PyTorch Dataset for BPR Loss ---")
# 1. Tạo một set chứa tất cả các item indices để lấy mẫu âm nhanh
all_item_indices = set(aid2idx.values())
print(f"Total unique item indices: {len(all_item_indices)}")

# 2. Tạo một dictionary: user_idx -> set(các item_idx đã tương tác)
# Cấu trúc này giúp kiểm tra nhanh xem một item có phải là negative hay không
print("Creating a map of user's positive items...")
# Chuyển đổi interactions sang index
interactions_indexed = interactions.with_columns([
    pl.col('session').replace(session2idx),
    pl.col('aid').replace(aid2idx)
])

# Gom nhóm lại
user_pos_items = interactions_indexed.group_by('session').agg(pl.col('aid'))
user_pos_items_dict = {row[0]: set(row[1]) for row in user_pos_items.rows()}

# Chuyển interactions thành list các tuple (user_idx, item_idx) để dùng trong Dataset
interaction_pairs = interactions_indexed.rows()

print(f"Created positive item map for {len(user_pos_items_dict)} users.")

--- Step 2: Building PyTorch Dataset for BPR Loss ---
Total unique item indices: 895417
Creating a map of user's positive items...
Created positive item map for 554926 users.


In [17]:
class BPRDataset(Dataset):
    def __init__(self, interaction_pairs, all_item_indices, user_pos_items_dict, num_aids):
        """
        Khởi tạo Dataset cho BPR Loss.
        
        Args:
            interaction_pairs (list): List các tuple (user_idx, positive_item_idx).
            all_item_indices (set): Set chứa tất cả các item index.
            user_pos_items_dict (dict): Dict: user_idx -> set(positive_item_indices).
            num_aids (int): Tổng số lượng item.
        """
        self.interaction_pairs = interaction_pairs
        self.all_item_indices = list(all_item_indices) # Chuyển sang list để có thể index
        self.user_pos_items = user_pos_items_dict
        self.num_aids = num_aids

    def __len__(self):
        return len(self.interaction_pairs)

    def __getitem__(self, index):
        # 1. Lấy một cặp tương tác dương (user, positive_item)
        user_idx, pos_item_idx = self.interaction_pairs[index]
        
        # 2. Lấy mẫu một item âm (negative_item)
        neg_item_idx = None
        while neg_item_idx is None or neg_item_idx in self.user_pos_items[user_idx]:
            # Lấy ngẫu nhiên một item từ toàn bộ danh sách
            neg_item_idx = random.choice(self.all_item_indices)
            
        return user_idx, pos_item_idx, neg_item_idx

In [18]:
import torch_geometric.nn as pyg_nn
from torch_geometric.utils import to_scipy_sparse_matrix

In [19]:

from torch_geometric.nn.conv import GCNConv # Import lớp GCNConv
from torch_geometric.nn.conv.gcn_conv import gcn_norm
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# ===================================================================
# Bước 3: ĐỊNH NGHĨA LẠI CLASS LIGHTGCN (Sử dụng GCNConv)
# ===================================================================
class LightGCN(nn.Module):
    def __init__(self, num_users, num_items, embed_dim=32, num_layers=3):
        super().__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.embedding = nn.Embedding(num_users + num_items, embed_dim)
        nn.init.xavier_uniform_(self.embedding.weight)
        
        # Chúng ta vẫn cần khởi tạo các lớp GCNConv,
        # mặc dù chỉ dùng phương thức propagate của chúng.
        self.convs = nn.ModuleList([GCNConv(embed_dim, embed_dim) for _ in range(num_layers)])

    def forward(self, edge_index):
        # Lấy embedding khởi tạo
        x0 = self.embedding.weight
        all_layer_embeddings = [x0]
        
        # Chuẩn hóa ma trận kề một lần duy nhất
        # Đây là bước quan trọng nhất để mô phỏng LightGCN
        norm_edge_index, norm_edge_weight = gcn_norm(
            edge_index, 
            num_nodes=self.num_users + self.num_items
        )

        x = x0
        for conv in self.convs:
            # GỌI TRỰC TIẾP `propagate` ĐỂ THỰC HIỆN PHÉP TỔNG HỢP HÀNG XÓM
            # mà không cần qua lớp Linear `conv.lin`
            x = conv.propagate(norm_edge_index, x=x, edge_weight=norm_edge_weight)
            all_layer_embeddings.append(x)
        
        final_embedding = torch.mean(torch.stack(all_layer_embeddings, dim=0), dim=0)
        
        users_emb, items_emb = torch.split(final_embedding, [self.num_users, self.num_items])
        
        return users_emb, items_emb

# --- Khởi tạo Model ---
model = LightGCN(num_users=num_sessions, num_items=num_aids, embed_dim=32, num_layers=3).to(DEVICE)
edge_index = edge_index.to(DEVICE)

In [20]:
model

LightGCN(
  (embedding): Embedding(1450343, 32)
  (convs): ModuleList(
    (0-2): 3 x GCNConv(32, 32)
  )
)

In [21]:
print(edge_index.shape)

torch.Size([2, 10239052])


In [22]:
users, items = model(edge_index)
print(users.shape)
print(items.shape)

torch.Size([554926, 32])
torch.Size([895417, 32])


In [ ]:
import torch.optim as optim
from torch.cuda.amp import autocast, GradScaler
import torch.nn.functional as F
import numpy as np
import os
from sklearn.model_selection import train_test_split # Dùng để chia dữ liệu

# --- CÁC THAM SỐ CẤU HÌNH ---
EPOCHS = 40
LEARNING_RATE = 1e-3
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
REAL_BATCH_SIZE = 2048
ACCUMULATION_STEPS = 2
EFFECTIVE_BATCH_SIZE = REAL_BATCH_SIZE * ACCUMULATION_STEPS
VALIDATION_SPLIT = 0.1 # Dành 10% dữ liệu cho validation

# Tạo thư mục output
OUTPUT_DIR = "/kaggle/working/lightgcn_output/"
os.makedirs(OUTPUT_DIR, exist_ok=True)
BEST_MODEL_PATH = os.path.join(OUTPUT_DIR, "lightgcn_best_model.pth")

# --- BƯỚC MỚI: CHIA DỮ LIỆU THÀNH TRAIN/VALIDATION ---
print(f"--- Splitting interaction data into train/validation ({1-VALIDATION_SPLIT:.0%}/{VALIDATION_SPLIT:.0%}) ---")
train_interactions, val_interactions = train_test_split(
    interaction_pairs,
    test_size=VALIDATION_SPLIT,
    random_state=42
)

# --- Khởi tạo Dataset & DataLoader cho cả Train và Valid ---
train_dataset = BPRDataset(train_interactions, all_item_indices, user_pos_items_dict, num_aids)
train_loader = DataLoader(train_dataset, batch_size=REAL_BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)

val_dataset = BPRDataset(val_interactions, all_item_indices, user_pos_items_dict, num_aids)
val_loader = DataLoader(val_dataset, batch_size=REAL_BATCH_SIZE, shuffle=False, num_workers=2) # Không cần shuffle validation set

print(f"Train samples: {len(train_dataset)}, Validation samples: {len(val_dataset)}")

# --- Khởi tạo Model và các thành phần Training ---
# (Đảm bảo model, edge_index đã được khởi tạo và chuyển sang DEVICE)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scaler = GradScaler()
# Hàm BPR Loss

def bpr_loss(users_emb, pos_items_emb, neg_items_emb):
    # Đảm bảo các tensor có ít nhất 2 chiều
    #print(users_emb.dim(), users_emb)
    if users_emb.dim() == 1: users_emb = users_emb.unsqueeze(0)
    if pos_items_emb.dim() == 1: pos_items_emb = pos_items_emb.unsqueeze(0)
    if neg_items_emb.dim() == 1: neg_items_emb = neg_items_emb.unsqueeze(0)
        
    pos_scores = torch.sum(users_emb * pos_items_emb, dim=1)
    neg_scores = torch.sum(users_emb * neg_items_emb, dim=1)
    
    return -torch.mean(F.logsigmoid(pos_scores - neg_scores)) # Dùng F.logsigmoid ổn định hơn


print(f"Starting training with Effective Batch Size: {EFFECTIVE_BATCH_SIZE}")
print(f"Best model will be saved to: {BEST_MODEL_PATH}")

# --- BIẾN ĐỂ THEO DÕI CHECKPOINT ---
best_val_loss = float('inf')

# --- VÒNG LẶP HUẤN LUYỆN VÀ ĐÁNH GIÁ ---
for epoch in range(EPOCHS):
    # --- GIAI ĐOẠN TRAINING ---
    model.train()
    loop_train = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [Training]")
    total_train_loss = 0
    
    optimizer.zero_grad()
    for batch_idx, (users, pos_items, neg_items) in enumerate(loop_train):
        # ... (toàn bộ logic training với gradient accumulation giữ nguyên như cũ) ...
        # ... (chỉ thay đổi tên biến loss để rõ ràng hơn) ...
        with autocast():
            users_emb_final, items_emb_final = model(edge_index)
            users_emb = users_emb_final[users.to(DEVICE)]
            pos_items_emb = items_emb_final[pos_items.to(DEVICE) - num_sessions]
            neg_items_emb = items_emb_final[neg_items.to(DEVICE) - num_sessions]
            
            train_loss = bpr_loss(users_emb, pos_items_emb, neg_items_emb)
            train_loss = train_loss / ACCUMULATION_STEPS
        
        scaler.scale(train_loss).backward()
        
        if (batch_idx + 1) % ACCUMULATION_STEPS == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            
        total_train_loss += train_loss.item() * ACCUMULATION_STEPS
        loop_train.set_postfix(train_loss=train_loss.item() * ACCUMULATION_STEPS)
        
    if (len(train_loader)) % ACCUMULATION_STEPS != 0:
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()

    avg_train_loss = total_train_loss / len(train_loader)

    # --- GIAI ĐOẠN VALIDATION ---
    model.eval()
    loop_val = tqdm(val_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [Validation]")
    total_val_loss = 0
    
    with torch.no_grad(): # Không cần tính gradient cho validation
        # Lan truyền một lần duy nhất cho toàn bộ epoch validation
        val_users_emb_final, val_items_emb_final = model(edge_index)
        
        for users, pos_items, neg_items in loop_val:
            users, pos_items, neg_items = users.to(DEVICE), pos_items.to(DEVICE), neg_items.to(DEVICE)
            
            val_users_emb = val_users_emb_final[users]
            val_pos_items_emb = val_items_emb_final[pos_items - num_sessions]
            val_neg_items_emb = val_items_emb_final[neg_items - num_sessions]
            
            val_loss = bpr_loss(val_users_emb, val_pos_items_emb, val_neg_items_emb)
            total_val_loss += val_loss.item()
            
    avg_val_loss = total_val_loss / len(val_loader)
    
    print(f"Epoch {epoch+1}/{EPOCHS} - Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")
    
    # --- LOGIC CHECKPOINTING (DỰA TRÊN VAL LOSS) ---
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        print(f"  ** New best model saved at epoch {epoch+1} with Val Loss: {best_val_loss:.4f} **")

print("\n--- Training Complete ---")
print(f"Finished. Best model saved at {BEST_MODEL_PATH} with validation loss: {best_val_loss:.4f}")

In [24]:
import pickle
import os
import polars as pl
import torch
import gc

print("\n--- Saving Final Assets ---")

# 1. Tải lại trọng số của model tốt nhất
print(f"Loading best model weights from {BEST_MODEL_PATH}")
# Trước tiên, cần đảm bảo model đã được khởi tạo với đúng kiến trúc
# (Nếu cell này chạy trong cùng notebook với cell training, model đã tồn tại)
model.load_state_dict(torch.load(BEST_MODEL_PATH))
model.eval() # Chuyển sang chế độ đánh giá

# 2. Trích xuất Item Embeddings từ model tốt nhất
print("Extracting final item embeddings...")
with torch.no_grad():
    # --- SỬA LỖI Ở ĐÂY ---
    # Thực hiện một forward pass cuối cùng và "bắt" cả hai kết quả trả về
    final_users_emb, final_items_emb = model(edge_index)
    
    # Chúng ta chỉ quan tâm đến `final_items_emb`
    final_items_emb_numpy = final_items_emb.cpu().numpy()

# 3. Lưu Item Embeddings ra file
# Lấy lại các aid theo đúng thứ tự index
# Logic này cần được sửa lại một chút cho an toàn
# Chúng ta sẽ tạo một DataFrame ánh xạ từ aid2idx để đảm bảo thứ tự
print("Creating final embedding DataFrame...")
aid_map_df = pl.DataFrame({
    'aid': list(aid2idx.keys()),
    'idx': list(aid2idx.values())
}).filter(pl.col('idx') >= num_sessions) # Chỉ lấy các item

# Sắp xếp theo đúng index (từ num_sessions trở đi)
aid_map_df = aid_map_df.sort('idx')
aids_in_order = aid_map_df['aid'].to_list()

# Kiểm tra lại kích thước
assert len(aids_in_order) == final_items_emb_numpy.shape[0], "Mismatch between number of AIDs and number of embeddings!"

embedding_df = pl.DataFrame({
    'aid': aids_in_order,
    'embedding': [list(emb) for emb in final_items_emb_numpy]
})

EMBEDDING_SAVE_PATH = os.path.join(OUTPUT_DIR, "item_embeddings_gnn.pqt")
embedding_df.write_parquet(EMBEDDING_SAVE_PATH)
print(f"Item embeddings saved to: {EMBEDDING_SAVE_PATH}")

# 4. Lưu lại các từ điển ánh xạ quan trọng
print("Saving mapping dictionaries...")
SESSION2IDX_PATH = os.path.join(OUTPUT_DIR, "session2idx.pkl")
with open(SESSION2IDX_PATH, 'wb') as f:
    pickle.dump(session2idx, f)
    
AID2IDX_PATH = os.path.join(OUTPUT_DIR, "aid2idx.pkl")
with open(AID2IDX_PATH, 'wb') as f:
    pickle.dump(aid2idx, f)

# (Lưu các từ điển ngược nếu cần)
IDX2AID_PATH = os.path.join(OUTPUT_DIR, "idx2aid.pkl")
with open(IDX2AID_PATH, 'wb') as f:
    pickle.dump(idx2aid, f)

# 5. Dọn dẹp

print("\n--- GNN Asset Generation Complete! ---")


--- Saving Final Assets ---
Loading best model weights from /kaggle/working/lightgcn_output/lightgcn_best_model.pth
Extracting final item embeddings...
Creating final embedding DataFrame...
Item embeddings saved to: /kaggle/working/lightgcn_output/item_embeddings_gnn.pqt
Saving mapping dictionaries...

--- GNN Asset Generation Complete! ---
